# TSA Chapter 2: Python Lab: Complete Residual Diagnostics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch2/TSA_ch2_python_diagnostics/TSA_ch2_python_diagnostics.ipynb)

This notebook demonstrates:
- Complete residual diagnostic workflow: residual time plot, ACF of residuals, Q-Q plot, histogram, Ljung-Box test, Jarque-Bera normality test

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
np.random.seed(42)

print("=" * 60)
print("PYTHON EXERCISE 3: RESIDUAL DIAGNOSTICS")
print("=" * 60)

# Generate ARMA(1,1) data and fit model
ar_params = np.array([1, -0.7])
ma_params = np.array([1, 0.4])
arma_process = ArmaProcess(ar_params, ma_params)
y = arma_process.generate_sample(nsample=300)

model = ARIMA(y, order=(1, 0, 1)).fit()
resid = model.resid

print(f"\nFitted ARMA(1,1) model")
print(f"Residual mean: {resid.mean():.6f}")
print(f"Residual std:  {resid.std():.6f}")

# Create diagnostic plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# 1. Residuals over time
axes[0, 0].plot(resid, color='steelblue', linewidth=0.8)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=0.8)
axes[0, 0].set_title('Residuals Over Time')
axes[0, 0].set_xlabel('t')

# 2. ACF of residuals
plot_acf(resid, ax=axes[0, 1], lags=20, alpha=0.05)
axes[0, 1].set_title('ACF of Residuals')

# 3. Q-Q plot
stats.probplot(resid, dist='norm', plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (Normality)')

# 4. Histogram of residuals
axes[1, 1].hist(resid, bins=30, density=True, color='steelblue', alpha=0.7, edgecolor='white')
x_range = np.linspace(resid.min(), resid.max(), 100)
axes[1, 1].plot(x_range, stats.norm.pdf(x_range, resid.mean(), resid.std()),
                'r-', linewidth=2, label='Normal')
axes[1, 1].set_title('Residual Distribution')
axes[1, 1].legend()

plt.tight_layout()
save_chart(fig, 'sem2_diagnostics')
plt.show()

# Ljung-Box test
lb_test = acorr_ljungbox(resid, lags=[5, 10, 15, 20], return_df=True)
print(f"\nLjung-Box Test Results:")
print(f"{'Lag':>5} {'Q-stat':>10} {'p-value':>10} {'Decision':>15}")
print("-" * 42)
for lag, row in lb_test.iterrows():
    decision = "White noise ✓" if row['lb_pvalue'] > 0.05 else "Autocorrelation ✗"
    print(f"{lag:>5} {row['lb_stat']:>10.4f} {row['lb_pvalue']:>10.4f} {decision:>15}")

# Jarque-Bera normality test
jb_stat, jb_pval = stats.jarque_bera(resid)
print(f"\nJarque-Bera Test: statistic = {jb_stat:.4f}, p-value = {jb_pval:.4f}")
if jb_pval > 0.05:
    print("  → Fail to reject H₀: residuals are normally distributed ✓")
else:
    print("  → Reject H₀: residuals are NOT normally distributed ✗")